In [77]:
from utils import geo

import pandas as pd
import rioxarray as rioxr
import xarray as xr
import geopandas as gpd
import sys
import yaml
import atlite
import glob
import os
import numpy as np

ROOT = 'C:/Users/49176/Uni/Masterthesis/'

In [78]:
files = glob.glob(ROOT + f'data/era5/{cntry}/' + '*.nc')
files = [os.path.basename(file) for file in files]

In [100]:
with open(ROOT + "/windgenpotential/countrycode.yaml", 'r') as stream:
    countrycode = yaml.safe_load(stream)  

countries = [i for i in countrycode.keys()]

In [ ]:
for cntry in countries: 

    cf = {}

    for n, wy in enumerate(files):

        cutout = atlite.Cutout(path = ROOT + f'data/era5/{cntry}/{wy}')

        if n == 0:

            ## 2. Get GWA data for mean correction
            # Import GWA Data for Germany
            gwa = rioxr.open_rasterio(ROOT + f'data/gwa/{cntry}_wind-speed_100m.tif')
            gwa = gwa.rename({'x' : 'lon', 'y' : 'lat'})

            # Move the spatial_ref to attributes
            gwa.attrs['spatial_ref'] = gwa['spatial_ref'].attrs
            gwa = gwa.drop(['spatial_ref'])

            # Drop the flat band dimension 
            gwa = gwa.squeeze('band')
            gwa = gwa.reset_coords('band', drop=True)

            # Interpolate -> downscale GWA grid to ERA5 resolution
            gwa_interpol = gwa.interp(lat=cutout.data.lat, lon=cutout.data.lon, method = 'linear')

        cf[wy] = (gwa_interpol / cutout.data['wnd100m'].mean('time')).values

        cf_mean = sum(cf.values()) / len(files)

    np.save(arr = cf_mean, file = ROOT + f'data/country_era5_gwa_cf/{cntry}.npy')

In [137]:
%%time
cntry = 'Germany'
scenario = 'base'
urban_distance = 1000
slope = 30
forest_share = 0
lpa_share = 0
turbine_1 = 'Vestas_V90_3MW'

try:
    turbine_2 = 'Vestas_V112_3MW'
except:
    turbine_2 = None

## 0. Get turbine_config .yaml
with open(ROOT + "windgenpotential/turbine.yaml", 'r') as stream:
    turbine_config = yaml.safe_load(stream)    
    

if turbine_2 != None:
    
    turbine_placement = gpd.read_file(ROOT + f'data/country_turbine_placements/{cntry}-{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{turbine_1}-{turbine_2}.gpkg')
    
    ## 2. Get GWA data for mean correction
    # Import GWA Data for Germany
    gwa_cf = np.load(ROOT + f'data/country_era5_gwa_cf/{cntry}.npy')
    
    files = glob.glob(ROOT + f'data/era5/{cntry}/' + '*.nc')
    files = [os.path.basename(file) for file in files]
    
    output={}
    
    for wy in files:
    
        ##1. Get y-cutout
        cutout = atlite.Cutout(path = ROOT + f'data/era5/{cntry}/{wy}')

        # Mean correction in data
        cutout.data['wnd100m'].values = cutout.data['wnd100m'].values * gwa_cf

        # Get the cell layout 
        cells = cutout.grid

        a = cells.drop(['x', 'y'], axis = 1)

        b = gpd.sjoin(a, turbine_placement[turbine_placement.ttype == 1].to_crs(4326) , how="left").drop('index_right', axis = 1).reset_index().dissolve(by = 'index', aggfunc='count')
        b['capacity'] = b['ttype'] * turbine_config[turbine_1].get('capacity')

        c = gpd.sjoin(a, turbine_placement[turbine_placement.ttype == 2].to_crs(4326) , how="left").drop('index_right', axis = 1).reset_index().dissolve(by = 'index', aggfunc='count')
        c['capacity'] = c['ttype'] * turbine_config[turbine_2].get('capacity')

        layout = xr.DataArray(b.capacity.values.reshape(cutout.shape), coords = cutout.data['wnd100m'].mean('time').coords).rename('Installed Capacity [MW]')
        generation_1 = cutout.wind(turbine=turbine_1, layout = layout).to_dataframe().droplevel(1)

        layout = xr.DataArray(c.capacity.values.reshape(cutout.shape), coords = cutout.data['wnd100m'].mean('time').coords).rename('Installed Capacity [MW]')
        generation_2 = cutout.wind(turbine=turbine_2, layout = layout).to_dataframe().droplevel(1)

        generation_1.rename(columns = {'specific generation' : turbine_1}, inplace = True)
        generation_2.rename(columns = {'specific generation' : turbine_2}, inplace = True)

        gen = pd.concat([generation_1, generation_2], axis = 1)
        
        output[wy] = gen
        
    output = pd.concat(output.values(), axis = 0)
        
    store = pd.HDFStore(ROOT + f'data/country_generation_potential_test/{cntry}.h5')
    store[f'{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{turbine_1}-{turbine_2}'] = output 
    store.close()

    
else: 

    turbine_placement = gpd.read_file(root + f'data/country_turbine_placements/{cntry}-{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{turbine_1}.gpkg')

    ## 2. Get GWA data for mean correction
    # Import GWA Data for Germany
    gwa_cf = np.load(ROOT + f'data/country_era5_gwa_cf/{cntry}.npy')
    
    files = glob.glob(ROOT + f'data/era5/{cntry}/' + '*.nc')
    files = [os.path.basename(file) for file in files]
    
    output={}
    
    for wy in files:
    
        ##1. Get y-cutout
        cutout = atlite.Cutout(path = ROOT + f'data/era5/{cntry}/{wy}')

        # Mean correction in data
        cutout.data['wnd100m'].values = cutout.data['wnd100m'].values * gwa_cf

        # Get the cell layout 
        cells = cutout.grid
    
    
        ## 3. Join the placed turbines and the era-grid to create the layout
        a = cells.drop(['x', 'y'], axis = 1)
        b = gpd.sjoin(a, turbine_placement.to_crs(4326) , how="left").drop('index_right', axis = 1).reset_index().dissolve(by = 'index', aggfunc='count')# .groupby('index_right').sum()
        b['capacity'] = b['ttype'] * turbine_config[turbine_1].get('capacity')

        #create layout
        layout = xr.DataArray(b.capacity.values.reshape(cutout.shape), coords = cutout.data['wnd100m'].mean('time').coords).rename('Installed Capacity [MW]')

        ## 4. Compute the generation timeseries

        gen = cutout.wind(turbine=turbine_1, layout = layout).to_dataframe().droplevel(1)
        gen.rename(columns = {'specific generation' : turbine_1}, inplace = True)
        
        output[wy] = gen
        
    output = pd.concat(output.values(), axis = 0)

    store = pd.HDFStore(root + f'data/country_generation_potential_test/{cntry}.h5')
    store[f'{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{turbine_1}'] = gen 
    store.close()

CPU times: total: 1min 33s
Wall time: 1min 28s


C:\Users\49176\anaconda3\envs\atlite\lib\site-packages\tables\path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'base-1000-30-0-0-Vestas_V90_3MW-Vestas_V112_3MW'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
